# predict

> Use chroma, labeled instances, an LLM to predict email classification

In [1]:
#| default_exp predict

In [142]:
#| export
from pathlib import Path
import json
from typing import List, Tuple
import time
from google.api_core.exceptions import ResourceExhausted
from ratelimit import sleep_and_retry

from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.document_loaders import DataFrameLoader

from classifier.schema import predict
from classifier.load import get_possible_labels, get_training_instances, get_idx
from classifier.process import BISON_MAXIMUM_INPUT_TOKENS
from classifier.chroma import get_or_make_chroma, merge_summaries_with_instances

In [88]:
from tqdm import tqdm
import pandas as pd

In [89]:
data_dir = Path("../data")
assert data_dir.exists()

Load chroma with embedded summaries of labeled emails

In [90]:
chroma = get_or_make_chroma(data_dir)

Load our test summaries

In [91]:
summary_path = data_dir / "summaries.csv"
assert summary_path.exists()
summaries = pd.read_csv(summary_path)
summaries.head(5)

,summary
0,The customer received an invoice from Cardina...
1,The customer received an email from the State...
2,The customer would like to place a new order ...
3,"The customer, State of Florida Next Gen, upda..."
4,The customer is inquiring about an order plac...


In [92]:
training_instances = list(get_training_instances())

In [93]:
chroma_document_frame = merge_summaries_with_instances(
    summaries,
    training_instances
)
chroma_document_frame.head(2)

,summary,BU,case_number,ACCOUNT_BUSINESS_UNIT__C,received_at,sfdc_subcategory,predicted_category,predicted_subcategory,record_type,probability,Accuracy_upd,Bin,label,email_subject,email_body
idx,,,,,,,,,,,,,,,
20775,The customer received an invoice from Cardina...,PD,3553288,a1G4z00000H6C4aEAF,2023-10-13T12:37:20,Billing Statements,Billing / Invoice,NaN,1,0.474032,Correct,4,Billing / Invoice,"Invoices 1 of 1 for 2057199110 , TEXAS INSTITU...","Dear Valued Customer, Your Cardinal Health in..."
46774,The customer received an email from the State...,PD,3658829,NaN,2023-11-29T20:25:47,Account updates,Billing / Invoice,NaN,1,0.566661,Incorrect,5,Account/Inquiry,Invoice status from State of Florida Next Gen,External Email â€“ Please use caution before o...


In [94]:
train_idx, test_idx = get_idx()

In [95]:
test_frame = chroma_document_frame.loc[test_idx, :]
test_frame.head(2)

,summary,BU,case_number,ACCOUNT_BUSINESS_UNIT__C,received_at,sfdc_subcategory,predicted_category,predicted_subcategory,record_type,probability,Accuracy_upd,Bin,label,email_subject,email_body
idx,,,,,,,,,,,,,,,
50493,- Customer inquiry about a possible overage o...,PD,3671369,NaN,2023-12-05T16:02:05,Account balance,Billing / Invoice,NaN,1,0.349415,Correct,3,Billing / Invoice,Customer Inquiry Possible overage CINI 5813233...,Hello Team\n\nCan you please reach out to the ...
13780,"The customer, Lauren Walsh from Sanofi, sent ...",SPD,3524801,NaN,2023-10-02T19:22:23,Order Placement,General Inquiry,NaN,2,0.732580,Correct,7,General Inquiry,new po,"Good afternoon, please reach out to acct #215..."


In [96]:
test_documents = DataFrameLoader(test_frame.reset_index(drop=False), page_content_column='summary').load()

## Prediction prompt

In [97]:
labels = get_possible_labels()
labels

['Order Processing',
 'Product Inquiry',
 'Account/Inquiry',
 'General Inquiry',
 'Returns',
 'Billing / Invoice',
 'Delivery',
 'Credits',
 'Order Discrepancy',
 'Pricing',
 'Program / Promotions']

In [124]:
#| export
EMAIL_LABEL_SEP = "|||"

LABEL_STR = """- Order Processing
- Product Inquiry
- Account/Inquiry
- General Inquiry
- Returns
- Billing / Invoice
- Delivery
- Credits
- Order Discrepancy
- Pricing
- Program / Promotions

"""

PREDICTION_PROMPT_TEMPLATE = """\
Our customer service team wants to classify emails so they can be sent to the right support team.
Here are the labels they use.

--LABELS--
""" + LABEL_STR + """

Below are a series of emails that have already been labeled, use their example to identify what label the final email should get.
Your answer must be one of the options in the --LABELS-- list.
Return only the label from the above list that you chose.

--EMAILS--
{examples}
EMAIL: {email} """ + f"{EMAIL_LABEL_SEP} LABEL: "

PREDICTION_PROMPT = PromptTemplate.from_template(PREDICTION_PROMPT_TEMPLATE)

In [125]:
PREDICTION_PROMPT

PromptTemplate(input_variables=['email', 'examples'], template='Our customer service team wants to classify emails so they can be sent to the right support team.\nHere are the labels they use.\n\n--LABELS--\n- Order Processing\n- Product Inquiry\n- Account/Inquiry\n- General Inquiry\n- Returns\n- Billing / Invoice\n- Delivery\n- Credits\n- Order Discrepancy\n- Pricing\n- Program / Promotions\n\n\n\nBelow are a series of emails that have already been labeled, use their example to identify what label the final email should get.\nYour answer must be one of the options in the --LABELS-- list.\nReturn only the label from the above list that you chose.\n\n--EMAILS--\n{examples}\nEMAIL: {email} ||| LABEL: ')

In [126]:
#| export
def filter_examples(examples: List[Document], idx: int) -> List[Document]:
    return [e for e in examples if int(e.metadata.get('idx')) != int(idx)]

In [127]:
example_retrieved_documents = chroma.similarity_search(
    test_documents[0].page_content, 
    k=3)
len(example_retrieved_documents)

3

In [128]:
len(filter_examples(example_retrieved_documents, test_documents[0].metadata['idx']))

3

In [129]:
len(chroma.get()['ids'])

900

In [130]:
#| export
def format_example(example: Document) -> str:
    return f"EMAIL: {example.page_content.strip()} {EMAIL_LABEL_SEP} LABEL: {example.metadata.get('label')}"


def make_prediction_prompt(
        email_summary: Document,
        chroma: Chroma,
        limit: int = None
) -> str:
    idx = email_summary.metadata.get('idx')
    k = 5
    prompt = None
    keep_stuffing = True
    max_k = len(chroma.get()['ids']) if limit is None else limit
    while keep_stuffing:
        examples = chroma.similarity_search(email_summary.page_content, k=k)
        examples = filter_examples(examples, idx)
        if len(examples) == 0:
            k += 5
            continue
        else:
            example_str = ""
            for e in examples:
                e_formatted = format_example(e)
                if len(example_str) == 0:
                    example_str = e_formatted
                else:
                    example_str = example_str + "\n" + e_formatted
                prompt = PREDICTION_PROMPT.format(
                    email=email_summary.page_content,
                    examples=example_str
                )
                if len(prompt) >= BISON_MAXIMUM_INPUT_TOKENS:
                    keep_stuffing = False
                    break
        k += 5
        if k >= max_k:
            keep_stuffing = False
    return prompt

In [131]:
example_prompt = make_prediction_prompt(
    test_documents[0], 
    chroma=chroma,
    limit=3
)
print(len(example_prompt))
print(example_prompt)

2305
Our customer service team wants to classify emails so they can be sent to the right support team.
Here are the labels they use.

--LABELS--
- Order Processing
- Product Inquiry
- Account/Inquiry
- General Inquiry
- Returns
- Billing / Invoice
- Delivery
- Credits
- Order Discrepancy
- Pricing
- Program / Promotions



Below are a series of emails that have already been labeled, use their example to identify what label the final email should get.
Your answer must be one of the options in the --LABELS-- list.
Return only the label from the above list that you chose.

--EMAILS--
EMAIL: - Item 5681788 (NABI-HB SF 312U/ML 5ML) is short by 8 units, resulting in a total shortage value of $5,846.48.
- The last time this item was counted was on 9/21/2023.
- The customer wants to check with the accounts listed in the attached file to inquire about a possible overage of this item.
- The item hasn't had many sales recently. ||| LABEL: Order Discrepancy
EMAIL: The customer is reporting a short

In [132]:
predict(example_prompt)

 Order Discrepancy

In [133]:
test_prompts = [
    make_prediction_prompt(s, chroma, 3) for s in tqdm(
        test_documents, 
        leave=False, 
        ncols=80)]

In [143]:
test_predictions = []

for p in tqdm(test_prompts, ncols=80, leave=False):
    sleep_time = 1
    try:
        p_prediction = predict(p)
    except ResourceExhausted:
        while True:
            try:
                p_prediction = predict(p)
                break
            except ResourceExhausted:
                time.sleep(sleep_time)
                sleep_time = sleep_time * 2
    test_predictions.append(p_prediction.text.strip())

In [144]:
len(test_predictions)

100

In [145]:
test_predictions[0]

'Order Discrepancy'

In [146]:
test_labels = [d.metadata.get('label') for d in test_documents]

In [147]:
pd.Series(test_predictions).value_counts()

Order Processing        38
Account/Inquiry         18
Order Discrepancy       13
Returns                  8
Billing / Invoice        6
Pricing                  4
Delivery                 3
Credits                  3
Product Inquiry          3
General Inquiry          3
Program / Promotions     1
Name: count, dtype: int64

In [148]:
pd.DataFrame(
    list(zip(
        test_predictions, 
        [d.metadata.get('label') for d in test_documents],
        [d.metadata.get('idx') for d in test_documents])),
    columns=['pred', 'label', 'idx']
).to_csv(data_dir / 'sample_predictions.csv', index=False)

In [149]:
#| hide
import nbdev; nbdev.nbdev_export()